## Import Libraries and Load Data

In [ ]:
# Add src to path
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print(f"✅ Project root: {project_root}")
print(f"✅ Added to path: {src_path}")


In [ ]:
# Visualize cached levels data using chart visualizer
import json
import pandas as pd
import plotly.graph_objects as go

def visualize_cached_data(symbol, df_segment, timeframe='15m', useLogScale=True):
    """
    Visualize cached levels data from parquet file
    
    Args:
        symbol: Trading pair symbol
        df_segment: DataFrame segment with OHLCV and levels_json data
        timeframe: Timeframe to visualize from the levels_json
        useLogScale: Whether to use logarithmic scale
    """
    
    # Create figure with candlestick data
    fig = go.Figure(data=[
        go.Candlestick(
            x=df_segment['datetime'],
            open=df_segment['open'],
            high=df_segment['high'],
            low=df_segment['low'],
            close=df_segment['close'],
            name='Candlestick',
            showlegend=False
        )
    ])
    
    # Process each row's levels data
    levels_data = json.loads(df_segment['levels_json'].iloc[-1])
    
    # Get data for the specified timeframe
    if timeframe in levels_data:
        tf_data = levels_data[timeframe]
        
        if isinstance(tf_data, dict):
            # Draw lines
            for line in tf_data.get('lines', []):
                if isinstance(line, list) and len(line) >= 4:
                    # Line format: [(t1, p1, v1, swing1), (t2, p2, v2, swing2), line_type, strength/volume]
                    pivot1, pivot2, line_type, strength = line[:4]
                    
                    # Unpack pivots (each pivot has 4 elements: timestamp, price, volume, swing_type)
                    t1, p1 = pivot1[0], pivot1[1]  # Extract timestamp and price only
                    t2, p2 = pivot2[0], pivot2[1]
                    
                    # Convert timestamps if needed
                    if isinstance(t1, str):
                        t1 = pd.Timestamp(t1)
                    if isinstance(t2, str):
                        t2 = pd.Timestamp(t2)
                    
                    # Set color based on line type
                    if 'vah_line' in str(line_type):
                        color = 'green'
                        width = 1
                    elif 'val_line' in str(line_type):
                        color = 'red'
                        width = 1
                    elif 'naked_poc_line' in str(line_type):
                        color = 'white'
                        width = 2
                    elif 'poc_line' in str(line_type):
                        color = 'lightgray'
                        width = 1
                    elif 'channel' in str(line_type):
                        color = 'cyan'
                        width = 2
                    elif 'zigzag' in str(line_type):
                        color = 'yellow'
                        width = 1
                    else:
                        color = 'orange'
                        width = 1
                    
                    strength_text = f" (s={strength})" if strength else ""
                    fig.add_trace(go.Scatter(
                        x=[t1, t2], 
                        y=[p1, p2], 
                        mode='lines', 
                        line=dict(color=color, width=width, dash='dash'), 
                        name=str(line_type) + strength_text, 
                        showlegend=False,
                        hovertemplate=f"{line_type}<br>%{{y:.2f}}<extra></extra>"
                    ))
            
            # Draw pivots
            for pivot in tf_data.get('pivots', []):
                if isinstance(pivot, list) and len(pivot) >= 4:
                    # Pivot format: (timestamp, price, volume, swing_type)
                    t, p, v, swing_type = pivot[:4]
                    
                    # Convert timestamp if needed
                    if isinstance(t, str):
                        t = pd.Timestamp(t)
                    
                    # Color based on swing type
                    pivot_colors = {
                        'HH': 'lime',
                        'HL': 'lightgreen',
                        'LH': 'salmon',
                        'LL': 'red',
                        None: 'gray'
                    }
                    color = pivot_colors.get(swing_type, 'gray')
                    
                    fig.add_trace(go.Scatter(
                        x=[t], 
                        y=[p], 
                        mode='markers+text',
                        marker=dict(color=color, size=8, line=dict(color='white', width=1)),
                        text=[swing_type or '?'],
                        textposition='top center',
                        textfont=dict(size=8, color='white'),
                        name=f'Pivot {swing_type}', 
                        showlegend=False,
                        hovertemplate=f"Pivot: {swing_type}<br>Price: %{{y:.2f}}<br>Time: %{{x}}<extra></extra>"
                    ))

    # Update layout
    fig.update_layout(
        title=f'{symbol} - {timeframe} - Cached Levels Visualizer ({len(df_segment)} candles)', 
        xaxis_title='Time', 
        yaxis_title='Price', 
        template='plotly_dark', 
        yaxis_type='log' if useLogScale else 'linear',
        height=800,
        xaxis=dict(fixedrange=False),
        yaxis=dict(fixedrange=False),
        showlegend=False,
        xaxis_rangeslider_visible=False
    )
    
    fig.show()
    return fig

print("✅ Chart visualizer function for cached data created!")


In [8]:
# Visualize different segments of the cached data
print("🎯 Visualizing cached levels data...")

import pandas as pd
import json
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 50)


## SYMBOL
symbol = 'ATOMUSDT'

# Load the checkpoint file
checkpoint_path = Path(f'data/levels_cache/{symbol}-15m-levels.parquet.checkpoint')
df = pd.read_parquet(checkpoint_path)

print(f"✅ Loaded checkpoint file: {checkpoint_path}")
print(f"📊 Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"📅 Date Range: {df['datetime'].min()} to {df['datetime'].max()}")
df.head()


# Get the last 500 candles for visualization
segment_size = 250
df = df.tail(segment_size) if segment_size > 0 else df

print(f"📊 Visualizing last {len(df)} candles:")
print(f"   Date range: {df['datetime'].iloc[0]} to {df['datetime'].iloc[-1]}")
print(f"   Price range: ${df['low'].min():,.2f} - ${df['high'].max():,.2f}")

# Check what timeframes are available in the last row
last_levels = json.loads(df['levels_json'].iloc[-1])
available_timeframes = list(last_levels.keys())
print(f"   Available timeframes: {available_timeframes}")

# Visualize each available timeframe
for tf in available_timeframes:
    print(f"\n🔍 Visualizing {tf} timeframe levels...")
    try:
        visualize_cached_data(symbol, df, timeframe=tf, useLogScale=True)
    except Exception as e:
        print(f"❌ Error visualizing {tf}: {e}")
        import traceback
        traceback.print_exc()

🎯 Visualizing cached levels data...
✅ Loaded checkpoint file: data\levels_cache\ATOMUSDT-15m-levels.parquet.checkpoint
📊 Shape: 40,000 rows × 9 columns
📅 Date Range: 2022-11-22T00:30:00 to 2024-01-12T17:30:00
📊 Visualizing last 250 candles:
   Date range: 2024-01-10T03:15:00 to 2024-01-12T17:30:00
   Price range: $9.34 - $10.95
   Available timeframes: ['M', 'W', 'D', '1h', '15m']

🔍 Visualizing M timeframe levels...



🔍 Visualizing W timeframe levels...



🔍 Visualizing D timeframe levels...



🔍 Visualizing 1h timeframe levels...



🔍 Visualizing 15m timeframe levels...
